<a href="https://colab.research.google.com/github/Dave5918/Dave5918/blob/main/Copy_of_fertilizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
data=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
inputs=data.columns[1:9]
target=data.columns[9]
inputs=data[inputs]
target=data[target]
numerical=inputs.select_dtypes(include=np.number).columns.tolist()
category=inputs.select_dtypes(include="object").columns.tolist()

# sns.heatmap(data[numerical].corr(),annot=True)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(inputs,target,test_size=0.2,random_state=42)
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
x_train[numerical]=scaler.fit_transform(x_train[numerical])
x_test[numerical]=scaler.fit_transform(x_test[numerical])
test[numerical]=scaler.fit_transform(test[numerical])


from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder(sparse_output=False,handle_unknown="ignore")
encoder.fit(data[category])
encoded=list(encoder.get_feature_names_out(category))
x_train[encoded]=encoder.transform(x_train[category])
x_test[encoded]=encoder.transform(x_test[category])
test[encoded]=encoder.transform(test[category])

from sklearn.ensemble import GradientBoostingClassifier
model=GradientBoostingClassifier(n_estimators=10,learning_rate=0.1,subsample=0.5,random_state=42)
# model.fit(x_train[numerical + encoded],y_train) # Moved fitting to the next cell
# test_pred=model.predict_proba(test[numerical + encoded]) # Moved prediction to the next cell
from sklearn.metrics import accuracy_score
# accuracy=accuracy_score(y_test,model.predict(x_test[numerical + encoded])) # Moved accuracy calculation to the next cell

In [20]:
from sklearn.preprocessing import LabelEncoder
target_encoder = LabelEncoder()
y_train_encoded = target_encoder.fit_transform(y_train) # Encode y_train
y_test_encoded = target_encoder.transform(y_test) # Encode y_test
model.fit(x_train[numerical + encoded], y_train_encoded) # Fit model with encoded target
test_pred_proba = model.predict_proba(test[numerical + encoded])
top_3_preds_indices = np.argsort(test_pred_proba, axis=1)[:, ::-1][:, :3]

# Map the indices back to the original fertilizer names
top_3_labels = np.array([target_encoder.classes_[i] for i in top_3_preds_indices.ravel()]).reshape(top_3_preds_indices.shape)

submission = pd.read_csv('sample_submission.csv')
submission = pd.DataFrame({
    'id': submission['id'],
    'Fertilizer Name': [' '.join(row) for row in top_3_labels] # Join strings directly
})

submission.to_csv('submission_real_names.csv', index=False)
print("✅ Submission file with real fertilizer names saved successfully!")
print(submission.head())

# Calculate accuracy with encoded y_test
accuracy = accuracy_score(y_test_encoded, model.predict(x_test[numerical + encoded]))
print(f"Accuracy: {accuracy}")

✅ Submission file with real fertilizer names saved successfully!
       id             Fertilizer Name
0  750000     14-35-14 10-26-26 20-20
1  750001  17-17-17 14-35-14 10-26-26
2  750002     20-20 10-26-26 14-35-14
3  750003  14-35-14 10-26-26 17-17-17
4  750004     20-20 17-17-17 14-35-14
Accuracy: 0.1643
